In [2]:
!python -m unittest tests.test_expressions


E
ERROR: test_expressions (unittest.loader._FailedTest.test_expressions)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_expressions
Traceback (most recent call last):
  File "/home/monierashraf/anaconda3/lib/python3.12/unittest/loader.py", line 137, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'tests.test_expressions'


----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)


In [ ]:
!python -m unittest tests.test_parser

...............F^C


In [ ]:
!find . -type f -name "*.py" | sort

./1.py
./2.py
./3.py
./__init__.py
./m3.py
./maiaas22222n.py
./main.py
./src/ast/ast_nodes.py
./src/ast/__init__.py
./src/executor/__init__.py
./src/grammar/__init__.py
./src/grammar/TrinoLexer.py
./src/grammar/TrinoParserListener.py
./src/grammar/TrinoParser.py
./src/grammar/TrinoParserVisitor.py
./src/__init__.py
./src/optimizer/__init__.py
./src/parser/__init__.py
./src/parser/match_recognize_extractor.py
./src/validator/__init__.py
./tests/__init__.py
./tests/test_ast.py
./tests/test_parser_edge_cases.py
./tests/test_parser.py
./tests/test_validator.py


In [ ]:
testing

In [ ]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Each test query below is designed to either produce a valid AST or raise an error
# with detailed information. These tests cover our validation improvements.

test_queries = [
    # 1. Error: DEFINE clause is present without a PATTERN clause.
    {
        "name": "Error: DEFINE requires PATTERN",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. Error: DEFINE variable not found in the PATTERN clause.
    {
        "name": "Error: DEFINE variable not in PATTERN",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (A B+ C)
                DEFINE D AS totalprice < PREV(totalprice)
            );
        """
    },
    # 3. Error: NEXT() used without a column reference.
    {
        "name": "Error: NEXT() missing column reference",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES NEXT(1) AS next_value
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 4. Error: Exclusion syntax disallowed with unmatched rows.
    {
        "name": "Error: Exclusion syntax with unmatched rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A {- B+ C -} D)
                DEFINE D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 5. Error: AFTER MATCH SKIP target variable missing.
    {
        "name": "Error: AFTER MATCH SKIP target missing",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                AFTER MATCH SKIP TO LAST X
                PATTERN (A B+ C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 6. Error: AFTER MATCH SKIP to first variable disallowed.
    {
        "name": "Error: AFTER MATCH SKIP to first variable disallowed",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                AFTER MATCH SKIP TO FIRST A
                PATTERN (A B+ C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Error: Malformed PATTERN clause (missing closing parenthesis).
    {
        "name": "Error: Malformed PATTERN clause",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                ONE ROW PER MATCH
                PATTERN (A B+ C
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Error: Malformed DEFINE clause (missing AS keyword).
    {
        "name": "Error: Malformed DEFINE clause",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE B totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Valid: Empty pattern produces empty measures (except MATCH_NUMBER).
    {
        "name": "Valid: Empty pattern produces empty measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    A.totalprice AS starting_price,
                    MATCH_NUMBER() AS match_id
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Valid: Complex pattern with unconventional whitespace and nested navigation.
    {
        "name": "Valid: Complex pattern with unconventional whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES PREV(  FIRST( A.totalprice , 3 ) , 2 ) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (  A   B+   C )
                DEFINE B AS    totalprice < PREV(  totalprice )
            );
        """
    },
    # 11. Valid: Aggregate functions with consistent variable references.
    {
        "name": "Valid: Aggregate with consistent variable references",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    avg(A.totalprice) AS avg_price,
                    count(A.*) AS count_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 12. Error: Aggregate function with inconsistent variable references.
    {
        "name": "Error: Aggregate with inconsistent variable references",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES max_by(A.totalprice, B.totalprice) AS invalid_agg
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 13. Valid: Pattern using quantifiers (e.g. {2,3}).
    {
        "name": "Valid: Pattern with quantifiers",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B{2,3} C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 14. Valid: Pattern with reluctant quantifiers.
    {
        "name": "Valid: Pattern with reluctant quantifiers",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+? C)
                DEFINE B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 15. Valid: PERMUTE syntax with a union variable.
    {
        "name": "Valid: PERMUTE syntax and union variable",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES LAST(U.totalprice) AS top_price
                ONE ROW PER MATCH
                PATTERN (PERMUTE(A, B, C))
                SUBSET U = (A, B)
                DEFINE A AS totalprice > 0,
                       B AS totalprice < 100,
                       C AS totalprice IS NOT NULL
            );
        """
    },
    # 16. Valid: Multiple SUBSET clauses.
    {
        "name": "Valid: Multiple SUBSET clauses",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    A.totalprice AS starting_price,
                    LAST(U1.totalprice) AS bottom_price,
                    LAST(U2.totalprice) AS top_price
                ONE ROW PER MATCH
                PATTERN (A B+ C D+)
                SUBSET U1 = (B, C), U2 = (C, D)
                DEFINE 
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice > 0
            );
        """
    },
    # 17. Valid: Case-insensitive identifiers.
    {
        "name": "Valid: Case-insensitive identifiers",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    A.totalprice AS starting_price
                PATTERN (a b+ c)
                DEFINE 
                    b AS totalprice < PREV(totalprice)
            );
        """
    },
    # 18. Valid: MATCH_NUMBER used with aggregates.
    {
        "name": "Valid: MATCH_NUMBER with aggregates",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES 
                    MATCH_NUMBER() AS match_id,
                    avg(A.totalprice) AS avg_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE 
                    B AS totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases. For valid queries, print the AST; for error cases, print the error message.
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        # The parser now raises detailed ParserError messages.
        print(f"Error: {e}")
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_type', metadata={'semantics': 'RUNNING'}, is_classifier=True, is_match_number=False), Measure(expression='MATCH_NUMBER()', alias='match_id', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=True), Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.

MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_type', metadata={'semantics': 'RUNNING'}, is_classifier=True, is_match_number=False), Measure(expression='MATCH_NUMBER()', alias='match_id', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=True), Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=RowsPerMatchClause(mode=ONE ROW PER MATCH),
  after_match_skip=None,
  pattern=PatternClause(pattern='A B+ C', metadata={'variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(va

In [ ]:
from src.parser.query_parser import parse_query


if __name__ == "__main__":
    sample_query = """
SELECT sum(x)  FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A  b+ c* )
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
    """
    components = parse_query(sample_query)
    print(components)

  

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT sum(x) FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A b+ c* ) DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=sum(x), metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression

FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=sum(x), metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
),
  metadata={}
)


In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

def run_test(query, description):
    print(f"--- Running Test: {description} ---")
    try:
        ast = parse_full_query(query)
        print("Test Passed: Query parsed successfully.")
        print(ast)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("="*80)

if __name__ == "__main__":
    # 1. Alternation using '|' operator
    test_query1 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A | B | C)
    DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100
);
"""
    run_test(test_query1, "Alternate forms: Alternation using '|' operator")

    # 2. Grouping
    test_query2 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B) C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query2, "Alternate forms: Grouping")

    # 3. Permutation
    test_query3 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A, B, C))
    SUBSET U = (A, B, C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query3, "Alternate forms: Permutation")

    # 4. Quantifiers with bounds (Greedy)
    test_query4a = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4})
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query4a, "Quantifiers with bounds: Greedy")

    # 4. Quantifiers with bounds (Reluctant)
    test_query4b = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4}?)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query4b, "Quantifiers with bounds: Reluctant")

    # 5. Valid Navigation and Aggregates
    test_query5 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(A.totalprice) AS final_price, avg(A.totalprice) AS avg_price
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query5, "Valid Navigation and Aggregates")

    # 6. Invalid Navigation Function (should fail)
    test_query6 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(1) AS invalid_nav
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query6, "Invalid Navigation Function (should fail)")

    # 7. Invalid Nested Aggregate in Navigation (should fail)
    test_query7 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES FIRST(avg(A.totalprice),2) AS invalid_agg_nav
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query7, "Invalid Nested Aggregate in Navigation (should fail)")

    # 8. Case-sensitive identifier validation
    test_query8 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    run_test(test_query8, "Case-sensitive identifier validation")

    # 9. Grouping with Quantifiers
    test_query9 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query9, "Alternate forms: Grouping with quantifiers")

    # 10. Aggregate with inconsistent variable references (should fail)
    test_query10 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES max_by(A.totalprice, B.totalprice) AS invalid_agg
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query10, "Aggregate with inconsistent variable references (should fail)")

    # 11. Aggregate with consistent variable references
    test_query11 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES avg(A.totalprice) AS avg_price, count(A.*) AS count_price
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query11, "Aggregate with consistent variable references")

    # 12. Navigation and Aggregate with MATCH_NUMBER and avg
    test_query12 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES MATCH_NUMBER() AS match_id, avg(A.totalprice) AS avg_price
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query12, "MATCH_NUMBER with aggregates")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A | B | C) DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price'

--- Running Test: Alternate forms: Alternation using '|' operator ---
Test Passed: Query parsed successfully.
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A | B | C', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>100'), Define(variable='B', condition='totalprice<50'), Define(variable='C', condition='totalpriceBETWEEN50AND100')])


In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

def run_test(query: str, description: str):
    print(f"--- Running Test: {description} ---")
    try:
        ast = parse_full_query(query)
        print("Test Passed: Query parsed successfully.")
        print(ast)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)

if __name__ == "__main__":
    # 1. Alternate forms: Alternation using '|' operator
    query1 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (A | B | C)
        DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100
    );
    """
    run_test(query1, "Alternate forms: Alternation using '|' operator")

    # 2. Alternate forms: Grouping (using parentheses)
    query2 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN ((A B C))
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query2, "Alternate forms: Grouping")

    # 3. Alternate forms: Permutation
    query3 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(U.totalprice) AS top_price
        PATTERN (PERMUTE(A, B, C))
        SUBSET U = (A, B, C)
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query3, "Alternate forms: Permutation")

    # 4a. Quantifiers with bounds: Greedy
    query4a = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
        PATTERN (A{2,4})
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query4a, "Quantifiers with bounds: Greedy")

    # 4b. Quantifiers with bounds: Reluctant
    query4b = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
        PATTERN (A{2,4}?)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query4b, "Quantifiers with bounds: Reluctant")

    # 5. Valid Navigation and Aggregates
    query5 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(A.totalprice) AS final_price, avg(A.totalprice) AS avg_price
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query5, "Valid Navigation and Aggregates")

    # 6. Invalid Navigation Function (should fail)
    query6 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(1) AS invalid_nav
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query6, "Invalid Navigation Function (should fail)")

    # 7. Invalid Nested Aggregate in Navigation (should fail)
    query7 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES FIRST(avg(A.totalprice),2) AS invalid_agg_nav
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query7, "Invalid Nested Aggregate in Navigation (should fail)")

    # 8. Case-Sensitive Identifier Validation
    query8 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (A b+ c)
        DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
    );
    """
    run_test(query8, "Case-Sensitive Identifier Validation")

    # 9. Grouping with Quantifiers
    query9 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN ((A B+ C+))
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query9, "Grouping with Quantifiers")

    # 10. Aggregate with inconsistent variable references (should fail)
    query10 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES max_by(A.totalprice, B.totalprice) AS invalid_agg
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice)
    );
    """
    run_test(query10, "Aggregate with Inconsistent Variable References (should fail)")

    # 11. Aggregate with consistent variable references
    query11 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES avg(A.totalprice) AS avg_price, count(A.*) AS count_price
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice), A AS TRUE, C AS TRUE
    );
    """
    run_test(query11, "Aggregate with Consistent Variable References")

    # 12. MATCH_NUMBER with aggregates
    query12 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES MATCH_NUMBER() AS match_id, avg(A.totalprice) AS avg_price
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice), A AS TRUE, C AS TRUE
    );
    """
    run_test(query12, "MATCH_NUMBER with Aggregates")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A | B | C) DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.tota

--- Running Test: Alternate forms: Alternation using '|' operator ---
Test Passed: Query parsed successfully.
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A | B | C', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>100'), Define(variable='B', condition='totalprice<50'), Define(variable='C', condition='totalpriceBETWEEN50AND100')])


In [ ]:
from src.parser.match_recognize_extractor import parse_match_recognize_query

# Define a list of test cases with a description and query string.
sample_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern without whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (Ab+c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Single word pattern (one token)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES vfggf.totalprice AS starting_price
    PATTERN (vfggf)
    DEFINE vfggf AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with PERMUTE construct",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A,B,C))
    SUBSET U = (A,B,C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with grouping and quantifiers",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Mixed pattern: whitespace vs. no whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with quantifiers (greedy)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4})
    DEFINE A AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with quantifiers (reluctant)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4}?)
    DEFINE A AS totalprice > 0
);
"""
    }
]

# Iterate over each test case.
for test in sample_queries:
    print(f"--- Running Test: {test['description']} ---")
    try:
        result = parse_match_recognize_query(test["query"])
        print("Test Passed: Query parsed successfully.")
        print(result)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']})
DEBUG:src.parser.match_recognize_extractor:Extracted DEFINE: DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
DEBUG:sr

--- Running Test: Simple pattern with whitespace ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Pattern without whitespace ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Single word pattern (one token) ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Pattern with PERMUTE construct ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Pattern with grouping and quantifiers ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Mixed pattern: whitespace vs. no whitespace ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Pattern with quantifiers (greedy) ---
Test Failed with error:
name 'ast' is not defined
--- Running Test: Pattern with quantifiers (reluctant) ---
Test Failed with error:
name 'ast' is not defined


In [8]:
from src.parser.match_recognize_extractor import parse_match_recognize_query

if __name__ == "__main__":
    sample_query = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A,B,C))
    SUBSET U = (A,B,C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);





    """


    components = parse_match_recognize_query(sample_query)
    print(components)


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='LAST(U.totalprice)', alias='top_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='PERMUTE(A,B,C)', metadata={'variables': ['ABC'], 'base_variables': ['ABC']})
DEBUG:src.parser.match_recognize_extractor:Extracted SUBSET: [SubsetClause(subset_text='U=(A,B,C)')]
DEBUG:src.parser.match_recognize_extractor:Extracted DEFINE: DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='B', condition=

MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='LAST(U.totalprice)', alias='top_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='PERMUTE(A,B,C)', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[SubsetClause(subset_text='U=(A,B,C)')],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='B', condition='totalprice<PREV(totalprice)'), Define(variable='C', condition='totalprice<100')])
)


In [ ]:
from src.parser.match_recognize_extractor import parse_match_recognize_query
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

test_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (A b+ c*)
                DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern without whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (AB+C)
                DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern with PERMUTE, grouping, and quantifiers",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES LAST(U.totalprice) AS top_price
                PATTERN (PERMUTE(A,B,C))
                SUBSET U = (A,B,C)
                DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
            );
        """
    },
    {
        "description": "Mixed pattern: whitespace vs. no whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (A b+ c*)
                DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern with quantifiers (greedy and reluctant)",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
                PATTERN (A{2,4}? )
                DEFINE A AS totalprice > 0
            );
        """
    },
]

def run_tests():
    for test in test_queries:
        print("\n--- Running Test: {} ---".format(test["description"]))
        try:
            ast = parse_match_recognize_query(test["query"])
            print("Test Passed: Query parsed successfully.")
            print(ast)
        except Exception as e:
            print("Test Failed with error:")
            print(e)
            print("Query:")
            print(test["query"])

if __name__ == "__main__":
    run_tests()


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']})
DEBUG:src.parser.match_recognize_extractor:Extracted DEFINE: DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
DEBUG:sr


--- Running Test: Simple pattern with whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
)

--- Running Test: Pattern without whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']

In [ ]:
test coverage

🔹 Running Test: Basic One Row Per Match
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: All Rows Per Match - Show Empty Matches
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: All Rows Per Match - Omit Empty Matches
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: All Rows Per Match - With Unmatched Rows
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Nested Pattern with Unmatched Rows
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Classifier and Match Number Functions
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Running vs Final Semantics in Measures
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Running Avg vs Final Count Functions
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Empty Match and Unmatched Rows
Error: name 'CustomErrorListener' is not defined
🔹 Running Test: Physical Navigation Functions
Error: name 'CustomErrorListener' is not def

In [ ]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Define a list of test queries to cover edge cases and full coverage
test_queries = [
    # 1. Basic ONE ROW PER MATCH
    {
        "name": "Basic One Row Per Match",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. ALL ROWS PER MATCH with SHOW EMPTY MATCHES, SUBSET clause and union variable
    {
        "name": "All Rows Per Match - Show Empty Matches",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price,
                    LAST(U.totalprice) AS top_price
                ALL ROWS PER MATCH SHOW EMPTY MATCHES
                PATTERN (A | B | C | D)
                SUBSET U = (C, D)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                    D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 3. ALL ROWS PER MATCH with OMIT EMPTY MATCHES
    {
        "name": "All Rows Per Match - Omit Empty Matches",
        "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                PARTITION BY account_id
                ORDER BY transaction_date
                MEASURES
                    A.amount AS start_amount,
                    C.amount AS final_amount
                ALL ROWS PER MATCH OMIT EMPTY MATCHES
                PATTERN (A | B | C)
                DEFINE
                    B AS amount < PREV(amount),
                    C AS amount > PREV(amount)
            );
        """
    },
    # 4. ALL ROWS PER MATCH WITH UNMATCHED ROWS
    {
        "name": "All Rows Per Match - With Unmatched Rows",
        "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                PARTITION BY region
                ORDER BY sale_date
                MEASURES
                    A.price AS initial_price,
                    D.price AS final_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A B C D)
                DEFINE
                    B AS price < PREV(price),
                    C AS price > PREV(price),
                    D AS price > A.price
            );
        """
    },
    # 5. Nested Pattern: union variables and repeated groups with WITH UNMATCHED ROWS
    {
        "name": "Nested Pattern with Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    F.totalprice AS ending_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A (B C | D E)+ F)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice = PREV(totalprice),
                    E AS totalprice >= PREV(totalprice)
            );
        """
    },
    # 6. Using Classifier() and MATCH_NUMBER() functions
    {
        "name": "Classifier and Match Number Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Running vs Final Semantics in MEASURES
    {
        "name": "Running vs Final Semantics in Measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Aggregates with Running vs Final semantics (avg and count)
    {
        "name": "Running Avg vs Final Count Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Empty Match: pattern that produces an empty match for every row
    {
        "name": "Empty Match and Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    count(A.*) AS match_count
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Physical Navigation Functions: PREV() and NEXT()
    {
        "name": "Physical Navigation Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(A.totalprice, 2) AS prev_price,
                    NEXT(A.totalprice, 1) AS next_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 11. Nested Navigation Function
    {
        "name": "Nested Navigation Function",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(FIRST(A.totalprice, 3), 2) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases and print the resulting AST
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 80)


In [ ]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Define a list of test queries to cover edge cases and full coverage
test_queries = [
    # 1. Basic ONE ROW PER MATCH
    {
        "name": "Basic One Row Per Match",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. ALL ROWS PER MATCH with SHOW EMPTY MATCHES, SUBSET clause and union variable
    {
        "name": "All Rows Per Match - Show Empty Matches",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price,
                    LAST(U.totalprice) AS top_price
                ALL ROWS PER MATCH SHOW EMPTY MATCHES
                PATTERN (A | B | C | D)
                SUBSET U = (C, D)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                    D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 3. ALL ROWS PER MATCH with OMIT EMPTY MATCHES
    {
        "name": "All Rows Per Match - Omit Empty Matches",
        "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                PARTITION BY account_id
                ORDER BY transaction_date
                MEASURES
                    A.amount AS start_amount,
                    C.amount AS final_amount
                ALL ROWS PER MATCH OMIT EMPTY MATCHES
                PATTERN (A | B | C)
                DEFINE
                    B AS amount < PREV(amount),
                    C AS amount > PREV(amount)
            );
        """
    },
    # 4. ALL ROWS PER MATCH WITH UNMATCHED ROWS
    {
        "name": "All Rows Per Match - With Unmatched Rows",
        "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                PARTITION BY region
                ORDER BY sale_date
                MEASURES
                    A.price AS initial_price,
                    D.price AS final_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A B C D)
                DEFINE
                    B AS price < PREV(price),
                    C AS price > PREV(price),
                    D AS price > A.price
            );
        """
    },
    # 5. Nested Pattern: union variables and repeated groups with WITH UNMATCHED ROWS
    {
        "name": "Nested Pattern with Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    F.totalprice AS ending_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A (B C | D E)+ F)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice = PREV(totalprice),
                    E AS totalprice >= PREV(totalprice)
            );
        """
    },
    # 6. Using Classifier() and MATCH_NUMBER() functions
    {
        "name": "Classifier and Match Number Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Running vs Final Semantics in MEASURES
    {
        "name": "Running vs Final Semantics in Measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Aggregates with Running vs Final semantics (avg and count)
    {
        "name": "Running Avg vs Final Count Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Empty Match: pattern that produces an empty match for every row
    {
        "name": "Empty Match and Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    count(A.*) AS match_count
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Physical Navigation Functions: PREV() and NEXT()
    {
        "name": "Physical Navigation Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(A.totalprice, 2) AS prev_price,
                    NEXT(A.totalprice, 1) AS next_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 11. Nested Navigation Function
    {
        "name": "Nested Navigation Function",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(FIRST(A.totalprice, 3), 2) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases and print the resulting AST
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price ONE ROW PER MATCH PATTERN (A B+ C) DEFINE B AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table=orders)
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression=A.total

🔹 Running Test: Basic One Row Per Match
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression=A.totalprice, alias=starting_price, metadata={'semantics': 'RUNNING'}, ), Measure(expression=LAST(B.totalprice), alias=bottom_price, metadata={'semantics': 'RUNNING'}, )]),
  rows_per_match=RowsPerMatchClause(mode=ONE ROW PER MATCH),
  after_match_skip=None,
  pattern=PatternClause(pattern=A B+ C, metadata={'variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable=B, condition=totalprice<PREV(totalprice))])
),
  metadata={}
)
🔹 Running Test: All Rows Per Match - Show Empty Matches
FullQueryAST(
  select_clause=SelectClause(items=[Sel

DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table=orders)
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression=A.totalprice, alias=starting_price, metadata={'semantics': 'RUNNING'}, ), Measure(expression=count(A.*), alias=match_count, metadata={'semantics': 'RUNNING'}, )])
DEBUG:src.parser.match_recognize_extractor:Extracted ROWS PER MATCH: RowsPerMatchClause(mode=ONE ROW PER MATCH)
DEBUG:src.parser.match_recogni

FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression=A.totalprice, alias=starting_price, metadata={'semantics': 'RUNNING'}, ), Measure(expression=count(A.*), alias=match_count, metadata={'semantics': 'RUNNING'}, )]),
  rows_per_match=RowsPerMatchClause(mode=ONE ROW PER MATCH),
  after_match_skip=None,
  pattern=PatternClause(pattern=(), metadata={'variables': []}),
  subset=[],
  define=None
),
  metadata={}
)
🔹 Running Test: Physical Navigation Functions
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(column

In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

def run_test_queries():
    test_queries = [
        {
            "description": "Mixed semantics: RUNNING, FINAL, and default (no prefix)",
            "query": """
                SELECT * FROM orders MATCH_RECOGNIZE(
                    PARTITION BY custkey
                    ORDER BY orderdate
                    MEASURES
                            RUNNING LAST(A.totalprice) AS starting_price,
         FINAL LAST(B.totalprice) AS bottom_price,
         C.totalprice AS top_price
                    ONE ROW PER MATCH
                    PATTERN (A B+ C)
                    DEFINE
                        B AS totalprice < PREV(totalprice),
                        C AS totalprice > PREV(totalprice)
                )
            """
        },
        {
            "description": "Default semantics (no prefix in measures)",
            "query": """
                SELECT * FROM orders MATCH_RECOGNIZE(
                    PARTITION BY custkey
                    ORDER BY orderdate
                    MEASURES
                        A.totalprice AS starting_price,
                        LAST(B.totalprice) AS bottom_price
                    ONE ROW PER MATCH
                    PATTERN (A B+)
                    DEFINE
                        B AS totalprice < PREV(totalprice)
                )
            """
        },
        {
            "description": "Mixed with extra spaces and explicit prefixes",
            "query": """
                SELECT * FROM orders MATCH_RECOGNIZE(
                    PARTITION BY custkey
                    ORDER BY orderdate
                    MEASURES
                        FINAL   A.totalprice   AS   starting_price,
                        RUNNING  LAST(B.totalprice)   AS  bottom_price,
                        RUNNING NEXT(C.totalprice) AS top_price
                    ALL ROWS PER MATCH
                    PATTERN ( A B+ C+ )
                    DEFINE
                        B AS totalprice < PREV(totalprice),
                        C AS totalprice > PREV(totalprice)
                )
            """
        },
        {
            "description": "Measures without aliases",
            "query": """
                SELECT * FROM orders MATCH_RECOGNIZE(
                    PARTITION BY custkey
                    ORDER BY orderdate
                    MEASURES
                        FINAL A.totalprice,
                        RUNNING LAST(B.totalprice)
                    ONE ROW PER MATCH
                    PATTERN (A B+)
                    DEFINE
                        B AS totalprice < PREV(totalprice)
                )
            """
        }
    ]

    for test in test_queries:
        print(f"---\nTest: {test['description']}")
        try:
            ast = parse_full_query(test['query'])
            print(ast)
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    run_test_queries()


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES RUNNING LAST(A.totalprice) AS starting_price, FINAL LAST(B.totalprice) AS bottom_price, C.totalprice AS top_price ONE ROW PER MATCH PATTERN (A B+ C) DEFINE B AS totalprice < PREV(totalprice), C AS totalprice > PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table=orders)
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)])
DEBUG:src.parser.match_recogniz

---
Test: Mixed semantics: RUNNING, FINAL, and default (no prefix)
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)]),
  measures=MeasuresClause(measures=[]),
  rows_per_match=RowsPerMatchClause(mode=ONE ROW PER MATCH),
  after_match_skip=None,
  pattern=PatternClause(pattern=A B+ C, metadata={'variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable=B, condition=totalprice<PREV(totalprice)), Define(variable=C, condition=totalprice>PREV(totalprice))])
),
  metadata={}
)
---
Test: Default semantics (no prefix in measures)
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecog

In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

if __name__ == "__main__":
    sample_query = """
    SELECT x ,b FROM orders MATCH_RECOGNIZE(
         PARTITION BY custkey
         ORDER BY orderdate 
         MEASURES
                  A.totalprice AS starting_price,
                  LAST(B.totalprice) AS bottom_price,
                  LAST(U.totalprice) AS top_price
         ALL ROWS PER MATCH OMIT EMPTY MATCHES
         
         PATTERN ( A | B | C | D )
         SUBSET U = (C, D)
         DEFINE
                  B AS totalprice < PREV(totalprice),
                  C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                  D AS totalprice > PREV(totalprice)
    )
    """
    query_ast = parse_full_query(sample_query)
    print(query_ast)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT x ,b FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price, LAST(U.totalprice) AS top_price ALL ROWS PER MATCH OMIT EMPTY MATCHES PATTERN ( A | B | C | D ) SUBSET U = (C, D) DEFINE B AS totalprice < PREV(totalprice), C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice, D AS totalprice > PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=x, metadata={}), SelectItem(expression=b, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table=orders)
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor

FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=x, metadata={}), SelectItem(expression=b, metadata={})]),
  from_clause=FromClause(table=orders),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression=A.totalprice, alias=starting_price, metadata={'semantics': 'RUNNING'}, ), Measure(expression=LAST(B.totalprice), alias=bottom_price, metadata={'semantics': 'RUNNING'}, ), Measure(expression=LAST(U.totalprice), alias=top_price, metadata={'semantics': 'RUNNING'}, )]),
  rows_per_match=RowsPerMatchClause(mode=ALL ROWS PER MATCH, OMIT EMPTY MATCHES),
  after_match_skip=None,
  pattern=PatternClause(pattern=A | B | C | D, metadata={'variables': ['A', 'B', 'C', 'D']}),
  subset=[SubsetClause(subset_text=U=(C,D))],
  define=DefineClause(definitions=[Define(variable=B, condition=

In [ ]:
from src.parser.match_recognize_extractor import parse_match_recognize_query
if __name__ == "__main__":
    sample_query = """
SELECT *
FROM orders
MATCH_RECOGNIZE(
  PARTITION BY custkey
  ORDER BY orderdate
  MEASURES
    CLASSIFIER() AS pattern_type,
    MATCH_NUMBER() AS match_id,
    A.totalprice AS starting_price,
    LAST(B.totalprice) AS bottom_price
  ONE ROW PER MATCH
  PATTERN (A B+ C)
  DEFINE
    B AS totalprice < PREV(totalprice)
);
    """
    components = parse_match_recognize_query(sample_query)

    print(components)

DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Detected CLASSIFIER function in measure: Measure(expression=CLASSIFIER(), alias=pattern_type, metadata={}, CLASSIFIER() detected)
DEBUG:src.parser.match_recognize_extractor:Detected MATCH_NUMBER function in measure: Measure(expression=MATCH_NUMBER(), alias=match_id, metadata={}, MATCH_NUMBER() detected)
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression=CLASSIFIER(), alias=pattern_type, metadata={}, CLASSIFIER() detected), Measure(expression=MATCH_NUMBER(), alias=match_id, metadata={}, MATCH_NUMBER() detected), Measure(expression=A.totalprice, alias=startin

MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column=orderdate, ordering=ASC, nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression=CLASSIFIER(), alias=pattern_type, metadata={}, CLASSIFIER() detected), Measure(expression=MATCH_NUMBER(), alias=match_id, metadata={}, MATCH_NUMBER() detected), Measure(expression=A.totalprice, alias=starting_price, metadata={}, ), Measure(expression=LAST(B.totalprice), alias=bottom_price, metadata={}, )]),
  rows_per_match=RowsPerMatchClause(mode=ONE ROW PER MATCH),
  after_match_skip=None,
  pattern=PatternClause(pattern=A B+ C, metadata={'variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable=B, condition=totalprice<PREV(totalprice))])
)


In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

if __name__ == "__main__":
    test_queries = [
        # 1️⃣ All Rows Per Match with Omit Empty Matches, Subset, and multiple DEFINE conditions
        {
            "name": "All Rows Per Match with Omit Empty Matches",
            "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                 PARTITION BY custkey
                 ORDER BY orderdate 
                 MEASURES
                          A.totalprice AS starting_price,
                          LAST(B.totalprice) AS bottom_price,
                          LAST(U.totalprice) AS top_price
                 ALL ROWS PER MATCH OMIT EMPTY MATCHES
                 PATTERN ( A | B | C | D )
                 SUBSET U = (C, D)
                 DEFINE
                          B AS totalprice < PREV(totalprice),
                          C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                          D AS totalprice > PREV(totalprice)
            )
            """
        },
        # 2️⃣ One Row Per Match with a quantifier pattern
        {
            "name": "One Row Per Match with Quantifier",
            "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                 PARTITION BY custkey
                 ORDER BY orderdate 
                 MEASURES
                          A.totalprice AS starting_price,
                          LAST(B.totalprice) AS bottom_price
                 ONE ROW PER MATCH
                 PATTERN ( A B+ C )
                 DEFINE B AS totalprice < PREV(totalprice)
            )
            """
        },
        # 3️⃣ All Rows Per Match with WITH UNMATCHED ROWS and a nested pattern
        {
            "name": "All Rows with Unmatched Rows and Nested Pattern",
            "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                 PARTITION BY custkey
                 ORDER BY orderdate 
                 MEASURES
                          A.totalprice AS starting_price,
                          F.totalprice AS ending_price
                 ALL ROWS PER MATCH WITH UNMATCHED ROWS
                 PATTERN ( A (B C | D E)+ F )
                 DEFINE 
                          B AS totalprice < PREV(totalprice),
                          C AS totalprice > PREV(totalprice),
                          D AS totalprice = PREV(totalprice),
                          E AS totalprice >= PREV(totalprice)
            )
            """
        },
        # 4️⃣ Exclusion Pattern
        {
            "name": "Exclusion Pattern",
            "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                 PARTITION BY account_id
                 ORDER BY transaction_date
                 MEASURES
                          A.amount AS first_amount,
                          C.amount AS last_amount
                 ONE ROW PER MATCH
                 PATTERN ( {- A -} B C )
                 DEFINE 
                          B AS amount < PREV(amount),
                          C AS amount > PREV(amount)
            )
            """
        },
        # 5️⃣ Complex DEFINE Conditions
        {
            "name": "Complex Conditions in DEFINE",
            "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                 PARTITION BY region
                 ORDER BY sale_date
                 MEASURES
                          A.price AS initial_price,
                          C.price AS final_price
                 ONE ROW PER MATCH
                 PATTERN ( A B C )
                 DEFINE 
                          B AS totalprice < PREV(totalprice) AND totalprice > 100,
                          C AS totalprice > PREV(totalprice) OR totalprice = 100
            )
            """
        },
        # 6️⃣ Pattern Permutation
        {
            "name": "Pattern Permutation",
            "query": """
            SELECT * FROM customers MATCH_RECOGNIZE(
                 PARTITION BY customer_id
                 ORDER BY transaction_date
                 MEASURES
                          A.amount AS first_spend,
                          C.amount AS final_spend
                 ONE ROW PER MATCH
                 PATTERN ( PERMUTE(A, B, C) )
                 DEFINE 
                          B AS amount < PREV(amount),
                          C AS amount > PREV(amount)
            )
            """
        },
        # 7️⃣ Range Quantifiers
        {
            "name": "Range Quantifiers",
            "query": """
            SELECT * FROM stock_data MATCH_RECOGNIZE(
                 PARTITION BY stock_symbol
                 ORDER BY trade_date
                 MEASURES
                          A.price AS first_price,
                          C.price AS final_price
                 ONE ROW PER MATCH
                 PATTERN ( A{2,4} B{3} C{,5} )
                 DEFINE 
                          B AS price < PREV(price),
                          C AS price > PREV(price)
            )
            """
        }
    ]

    # 🚀 Run all test cases and print the resulting ASTs
    for test in test_queries:
        print(f"🔹 Running Test: {test['name']}")
        query_ast = parse_full_query(test["query"])
        print(query_ast)
        print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price, LAST(U.totalprice) AS top_price ALL ROWS PER MATCH OMIT EMPTY MATCHES PATTERN ( A | B | C | D ) SUBSET U = (C, D) DEFINE B AS totalprice < PREV(totalprice), C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice, D AS totalprice > PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_

🔹 Running Test: All Rows Per Match with Omit Empty Matches


ParserError: All pattern variables must be explicitly defined. Missing definitions for: {'A', '|'} (Line: 1, Column: 14)
Snippet: ordersMATCH_RECOGNIZE(PARTITIONBYcustkeyORDERBYorderdateMEASURESA.totalpriceASstarting_price,LAST(B.totalprice)ASbottom_price,LAST(U.totalprice)AStop_priceALLROWSPERMATCHOMITEMPTYMATCHESPATTERN(A|B|C|D)SUBSETU=(C,D)DEFINEBAStotalprice<PREV(totalprice),CAStotalprice>PREV(totalprice)ANDtotalprice<=A.totalprice,DAStotalprice>PREV(totalprice))

In [ ]:
from src.parser.match_recognize_extractor import parse_full_query

if __name__ == "__main__":
    test_queries = [
        {
            "name": "Classifier and Match Number Functions",
            "query": """
            SELECT * FROM orders 
            MATCH_RECOGNIZE(
                PARTITION BY custkey 
                ORDER BY orderdate 
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH 
                PATTERN ( A B+ C ) 
                DEFINE 
                    B AS totalprice < PREV(totalprice)
            );
            """
        },
        {
            "name": "Running vs Final Semantics in MEASURES",
            "query": """
            SELECT * FROM orders 
            MATCH_RECOGNIZE(
                PARTITION BY custkey 
                ORDER BY orderdate 
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH 
                PATTERN ( A B+ C ) 
                DEFINE 
                    B AS totalprice < PREV(totalprice)
            );
            """
        },
        {
            "name": "Running Avg vs Final Count Functions",
            "query": """
            SELECT * FROM orders 
            MATCH_RECOGNIZE(
                PARTITION BY custkey 
                ORDER BY orderdate 
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH 
                PATTERN ( A B+ C ) 
                DEFINE 
                    B AS totalprice < PREV(totalprice)
            );
            """
        }
    ]
    
    for test in test_queries:
        print(f"🔹 Running Test: {test['name']}")
        query_ast = parse_full_query(test["query"])
        print(query_ast)
        print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES CLASSIFIER() AS pattern_type, MATCH_NUMBER() AS match_id, A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price ONE ROW PER MATCH PATTERN ( A B+ C ) DEFINE B AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause via robust splitting: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause via regex: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extra

🔹 Running Test: Classifier and Match Number Functions
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_type', metadata={'semantics': 'RUNNING'}, is_classifier=True, is_match_number=False), Measure(expression='MATCH_NUMBER()', alias='match_id', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=True), Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=RowsP

In [ ]:
import sys
import pandas as pd
# Use an absolute import for match_recognize.
from transformations.match_recognize import match_recognize

query = """
    SELECT id, name FROM employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
        {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    
try:
        output_df = match_recognize(query, pd.DataFrame(data))
        print("Match Recognize Output:")
        print(output_df)
except Exception as e:
        print(f"Error: {str(e)}")

ImportError: cannot import name 'AutomatonBuilder' from partially initialized module 'transformations.automaton.automaton_builder' (most likely due to a circular import) (/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_builder.py)